In [1]:
import os
import cobra
from cobra.core import Reaction
from cobra.flux_analysis.parsimonious import add_pfba
import pandas as pd
# currently working with local medusa installation -- anytime changes are made to medusa, we need to run installation
# again within the virtualenv by running setup.py
#import medusa
from medusa.reconstruct.expand.expand import iterative_gapfill_from_binary_phenotypes
import medusa
from medusa.flux_analysis import flux_balance

/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def load_universal_modelseed():
    seed_rxn_table = pd.read_csv('../data/reactions_seed_20180809.tsv',sep='\t')
    seed_rxn_table['id'] = seed_rxn_table['id'] + '_c'
    universal = cobra.io.load_json_model('../data/universal_mundy.json')
    # remove any reactions from the universal that don't have "OK" status
    # in modelSEED (guards against mass and charge-imbalanced reactions)
    ok_ids = list(seed_rxn_table.loc[(seed_rxn_table['status'] == 'OK') | (seed_rxn_table['status'] == 'HB')]['id'])
    remove_rxns = []
    for reaction in universal.reactions:
        if reaction.id not in ok_ids:
            remove_rxns.append(reaction)
    universal.remove_reactions(remove_rxns)
    # remove metabolites from the universal that are no longer present in any
    # reactions.
    mets_in_reactions = []
    for reaction in universal.reactions:
        mets = [met.id for met in reaction.metabolites]
        mets_in_reactions.extend(mets)
    mets_in_reactions = set(mets_in_reactions)

    mets_missing_reactions = []
    for metabolite in universal.metabolites:
        if metabolite.id not in mets_in_reactions:
            mets_missing_reactions.append(metabolite)
    universal.remove_metabolites(mets_missing_reactions)

    universal.repair()
    return universal


In [3]:
master_universal = load_universal_modelseed()

In [ ]:
# Load the biolog composition to be used for gapfilling
biolog_base_composition = pd.read_csv('../data/biolog_base_composition.csv',sep=',')
biolog_base_dict = dict(zip(biolog_base_composition['ID'],\
                          [1000 for i in range(0,len(biolog_base_composition['ID']))]))
# The biolog growth file has already been filtered by species that meet
# the minimum carbon source requirement, so we can use the entire dataframe
biolog_thresholded = pd.read_csv('../data/plata_thresholded.csv',sep='\t',index_col=0)

# get the list of ensembles already generated.
already_generated = os.listdir('../results/ensembles/')
# remove the .json extension to just get the name for each species
already_generated = [s.split('.')[0] for s in already_generated]

# Exclude species for which there is no feasible solution
# using this reaction bag (identified during previous iterations
# of this analysis)
exclude_species = ["Brachybacterium faecium"]


# Iterate over each species and generate and ensemble for each
for species_file in os.listdir('../data/modelseed_models/'): 
    
    # Load the species model. only continue if the species is in the filtered
    # biolog dataframe (i.e. it met our filtering criteria)
    species_name = species_file.split('.')[0]
    if (species_name in biolog_thresholded.index) and (
        species_name not in already_generated) and (
        species_name not in exclude_species):
        print("Building ensemble for " + species_name)
        model = cobra.io.load_json_model('../data/modelseed_models/' + species_file)

        # extract the biolog conditions for the model of interest
        mod_pheno = biolog_thresholded.loc[species_name]
        mod_pheno = list(mod_pheno[mod_pheno == True].index)

        # generate a fresh universal for each species
        universal = master_universal.copy()

        # check for biolog base components in the model. Add exchange reactions
        # if none exist and add the metabolite to the model if it does not
        # already exist
        add_mets = []
        add_exchanges = []
        for met in list(biolog_base_dict.keys()):
            try:
                model.metabolites.get_by_id(met)
            except:
                print('no '+met)
                add_met = universal.metabolites.get_by_id(met).copy()
                add_mets.append(add_met)

        model.add_metabolites(add_mets)

        for met in list(biolog_base_dict.keys()):
            # Search for exchange reactions
            try:
                model.reactions.get_by_id('EX_'+met)
            except:
                add_met = model.metabolites.get_by_id(met)
                ex_rxn = Reaction('EX_' + met)
                ex_rxn.name = "Exchange reaction for " + met
                ex_rxn.lower_bound = -1000
                ex_rxn.upper_bound = 1000
                ex_rxn.add_metabolites({add_met:-1})
                add_exchanges.append(ex_rxn)

        model.add_reactions(add_exchanges)

        # Find metabolites from the biolog data that are missing in the model
        # and add them from the universal
        missing_mets = []
        missing_exchanges = []
        media_dicts = {}
        for met_id in mod_pheno:
            try:
                model.metabolites.get_by_id(met_id)
            except:
                print(met_id + " was not in model, adding met and exchange reaction")
                met = universal.metabolites.get_by_id(met_id).copy()
                missing_mets.append(met)
                ex_rxn = Reaction('EX_' + met_id)
                ex_rxn.name = "Exchange reaction for " + met_id
                ex_rxn.lower_bound = -1000
                ex_rxn.upper_bound = 1000
                ex_rxn.add_metabolites({met:-1})
                missing_exchanges.append(ex_rxn)
            media_dicts[met_id] = biolog_base_dict.copy()
            media_dicts[met_id] = {'EX_'+k:v for k,v in media_dicts[met_id].items()}
            media_dicts[met_id]['EX_'+met_id] = 1000
        model.add_metabolites(missing_mets)
        model.add_reactions(missing_exchanges)

        # identify transporters for each biolog component in the universal model
        # and pick one that will enable transport in the gapfilling problem.
        transporters_in_universal = [rxn for rxn in universal.reactions if len(rxn.compartments)>1]
        for met in media_dicts.keys():
            metabolite = model.metabolites.get_by_id(met)
            base_met_id = met.split('_')[0]
            rxns_with_metabolite = metabolite.reactions
            transport = False
            for rxn in rxns_with_metabolite:
                metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
                if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
                    transport = True

            pick_transporter = {}
            if not transport:
                print("missing transporter for " + metabolite.name)
                for rxn in transporters_in_universal:
                    metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
                    if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
                        pick_transporter[met] = rxn.id

        # Add the transporters to the model
        transporters_to_add = list(pick_transporter.values())
        transporter_list = []
        for rxn in transporters_to_add:
            transporter_list.append(universal.reactions.get_by_id(rxn).copy())
        model.add_reactions(transporter_list)

        # remove the added transporters from the universal model
        universal.remove_reactions([universal.reactions.get_by_id(rxn) for rxn in transporters_to_add])

        # generate the ensemble for this species
        num_cycles = 100
        lower_bound = 0.05
        ensemble = iterative_gapfill_from_binary_phenotypes(\
                                         model,\
                                         universal,\
                                         media_dicts,\
                                         num_cycles,\
                                         lower_bound=lower_bound,\
                                         inclusion_threshold=1E-11,\
                                         exchange_reactions=False,\
                                         demand_reactions=False,\
                                         exchange_prefix='EX')

        # save the ensemble by pickling it
        ensemble.to_pickle('../results/ensembles/'+species_name+'.pickle')

Building ensemble for Streptococcus mitis
no cpd10515_e
no cpd00268_e
cpd11594_e was not in model, adding met and exchange reaction
cpd01133_e was not in model, adding met and exchange reaction
cpd00382_e was not in model, adding met and exchange reaction
cpd03198_e was not in model, adding met and exchange reaction
cpd00492_e was not in model, adding met and exchange reaction
cpd00232_e was not in model, adding met and exchange reaction
cpd19001_e was not in model, adding met and exchange reaction
cpd00709_e was not in model, adding met and exchange reaction
cpd00079_e was not in model, adding met and exchange reaction
cpd00142_e was not in model, adding met and exchange reaction
cpd00029_e was not in model, adding met and exchange reaction
missing transporter for Dextrin
missing transporter for N-Acetyl-D-mannosamine
missing transporter for TRHL
missing transporter for Sucrose
missing transporter for Stachyose
missing transporter for beta D-Galactose
missing transporter for Acetate
m

In [ ]:
ex_rxns = [rxn for rxn in ensemble.base_model.reactions \
                        if rxn.id.startswith('EX_')]
for source in media_dicts.keys():
    # close all exchange reactions
    for rxn in ex_rxns:
        rxn.lower_bound = 0
    #ensemble.base_model.medium = media_dicts[source]
    for ex_rxn in media_dicts[source].keys():
                    ensemble.base_model.reactions.get_by_id(ex_rxn).lower_bound = \
                        -1.0*media_dicts[source][ex_rxn]
                    ensemble.base_model.reactions.get_by_id(ex_rxn).upper_bound = \
                        1.0*media_dicts[source][ex_rxn]
    for member in ensemble.members:
        ensemble.set_state(member)
        # member should produce the minimum amount of required biomass
        # flux or more
        if ensemble.base_model.optimize().f > 0.001:
            print(member.id,source)
        else:
            print("no growth for ",member.id,source)